In [26]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

%matplotlib inline
# import seaborn as sns
#import models from scikit learn module:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC

In [3]:
af = pd.read_csv("../data/smartphone-activity/attributes.csv")
type(af)
df = pd.read_csv("../data/smartphone-activity/dataset.csv")
type(df)

pandas.core.frame.DataFrame

In [4]:
pd.set_option("display.max.columns",None)
af.head()


,name,type,description
0,feature_1,float,tBodyAcc-mean()-X
1,feature_2,float,tBodyAcc-mean()-Y
2,feature_3,float,tBodyAcc-mean()-Z
3,feature_4,float,tBodyAcc-std()-X
4,feature_5,float,tBodyAcc-std()-Y


In [5]:
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,feature_342,fea

In [6]:
from pyspark.sql import SparkSession  
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, FloatType

print ("HELLLO")
def get_field(t: str):
    if t == "float":
        return FloatType()
    elif t == "integer":
        return IntegerType()
    else:
        raise Exception("Not expected: %s" % t)

struct_fields = []
for index, row in af.iterrows():
    if row['name'] != "activity":
        struct_fields.append(StructField(row['name'], get_field(row['type'])))

schema = StructType(struct_fields)



HELLLO


In [24]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example: Reading CSV file without mentioning schema") \
    .getOrCreate()
 
dataset = spark.read.load("../data/smartphone-activity/dataset.csv",format="csv", sep=",", inferSchema=True, header=True)#, sep=",", schema=schema)
dataset.show()
dataset.schema

+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-------

In [54]:
attrs = spark.read.csv("../data/smartphone-activity/attributes.csv", header=True)
attrs.show()
featureNames = []
for singletonList in attrs.select("name").toPandas().values:
    for name in singletonList:
        if name.startswith("feature_"):
            featureNames.append(name)

assembler = VectorAssembler(
    inputCols=featureNames,
    outputCol="features")

training = assembler.transform(dataset)
training.show()

+----------+-----+-------------------+
|      name| type|        description|
+----------+-----+-------------------+
| feature_1|float|  tBodyAcc-mean()-X|
| feature_2|float|  tBodyAcc-mean()-Y|
| feature_3|float|  tBodyAcc-mean()-Z|
| feature_4|float|   tBodyAcc-std()-X|
| feature_5|float|   tBodyAcc-std()-Y|
| feature_6|float|   tBodyAcc-std()-Z|
| feature_7|float|   tBodyAcc-mad()-X|
| feature_8|float|   tBodyAcc-mad()-Y|
| feature_9|float|   tBodyAcc-mad()-Z|
|feature_10|float|   tBodyAcc-max()-X|
|feature_11|float|   tBodyAcc-max()-Y|
|feature_12|float|   tBodyAcc-max()-Z|
|feature_13|float|   tBodyAcc-min()-X|
|feature_14|float|   tBodyAcc-min()-Y|
|feature_15|float|   tBodyAcc-min()-Z|
|feature_16|float|     tBodyAcc-sma()|
|feature_17|float|tBodyAcc-energy()-X|
|feature_18|float|tBodyAcc-energy()-Y|
|feature_19|float|tBodyAcc-energy()-Z|
|feature_20|float|   tBodyAcc-iqr()-X|
+----------+-----+-------------------+
only showing top 20 rows

+---------+---------+---------+-------

In [58]:
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol="features", labelCol="activity")
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
# Specify multiple Params.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # type: ignore

# You can combine paramMaps, which are python dictionaries.
# Change output column name
paramMap2 = {lr.probabilityCol: "myProbability"}  # type: ignore
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)  # type: ignore

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["activity", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)
result = prediction.select("features", "activity", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, activity=%s -> prob=%s, prediction=%s"
          % (row.features, row.activity, row.myProbability, row.prediction))

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: activity)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under b

Py4JJavaError: An error occurred while calling o1303.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 202.0 failed 1 times, most recent failure: Lost task 3.0 in stage 202.0 (TID 620) (DE-C-000N0.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.com

In [60]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
print(kmeans)

model = kmeans.fit(training)

# Make predictions
predictions = model.transform(training)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

KMeans_54e0448e6873
Silhouette with squared euclidean distance = 0.25656861824573984
Cluster Centers: 
[ 2.74429686e-01 -1.89812999e-02 -1.08288513e-01 -9.66823470e-01
 -9.53959029e-01 -9.57494638e-01 -9.69024785e-01 -9.54777542e-01
 -9.57208346e-01 -9.13617602e-01 -5.48454831e-01 -7.95326136e-01
  8.16143470e-01  6.71097446e-01  8.24801720e-01 -9.51769545e-01
 -9.97097623e-01 -9.93666110e-01 -9.89850531e-01 -9.72822964e-01
 -9.63730400e-01 -9.58039353e-01 -3.60730334e-01 -5.41652661e-01
 -4.69164922e-01 -8.48085989e-03 -8.69787557e-03  9.61396560e-03
  4.74788417e-02  1.86655610e-01 -1.04564583e-01  1.98463197e-01
  8.00650986e-03  2.21777162e-01 -8.91576429e-02  1.12621482e-01
 -1.04052939e-01  7.25755994e-02 -2.91425518e-01 -2.36420910e-01
 -3.54419525e-01  6.19633991e-01  5.58204734e-01 -9.54150936e-01
 -9.70632828e-01 -9.61239555e-01 -9.55548963e-01 -9.71406222e-01
 -9.62315781e-01 -4.03609808e-01  5.84727820e-01  5.53500253e-01
 -3.19288467e-01  6.28378923e-01  5.50069525e-01  1.

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = dataset

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))